In [1]:
from pyspark.sql import SparkSession 
from pyspark.sql.functions import explode,col,expr,when
from pyspark.sql.types import ArrayType, IntegerType, ShortType
import pandas as pd


In [2]:
spark = SparkSession.builder.appName('task_3').getOrCreate()

25/04/29 17:28:08 WARN Utils: Your hostname, nischit-baral resolves to a loopback address: 127.0.1.1; using 10.10.42.113 instead (on interface enp2s0)
25/04/29 17:28:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/29 17:28:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/04/29 17:28:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
spark

In [4]:
df_pyspark = spark.read.option('multiline','true').json('works.json')


In [5]:
# df_pyspark.show()
df_pyspark.printSchema()

root
 |-- provider_group_id: long (nullable = true)
 |-- provider_groups: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- npi: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- tin: struct (nullable = true)
 |    |    |    |-- type: string (nullable = true)
 |    |    |    |-- value: string (nullable = true)



In [6]:
provider_group = df_pyspark.withColumn("new_provider", explode("provider_groups"))
providers_n = provider_group.withColumn("npi_exp", explode("new_provider.npi"))


provider_again = providers_n.select(
    "provider_group_id",
    col("npi_exp").alias("npi"),
    col("new_provider.tin.type").alias("tin_type"),
    col("new_provider.tin.value").alias("tin")
)         

provider_again.show()

+-----------------+----------+--------+----------+
|provider_group_id|       npi|tin_type|       tin|
+-----------------+----------+--------+----------+
|         10001001|1235233008|     ein|04-3267217|
|         10001001|1316041189|     ein|04-3267217|
|         10001001|1780788554|     ein|04-3267217|
|         10001001|1891068409|     ein|04-3267217|
|         10001001|1366459570|     ein|11-1562701|
|         10001001|1417915653|     ein|11-3358535|
|         10001001|1417915653|     ein|13-3888838|
|         10002001|1609829761|     ein|00-0004110|
|         10002001|1821482241|     ein|00-0004110|
|         10002001|1760986277|     ein|00-6980743|
|         10002001|1215075882|     ein|01-0550744|
|         10002001|1013917665|     ein|01-0555304|
|         10002001|1679780811|     ein|01-0555483|
|         10002001|1700093952|     ein|01-0555483|
|         10002001|1780072447|     ein|01-0555483|
|         10002001|1952532970|     ein|01-0555483|
|         10002001|1376647511| 

In [7]:
rem_hyp = provider_again.withColumn('tin', expr("replace(tin,'-','')"))
rem_hyp.show()

+-----------------+----------+--------+---------+
|provider_group_id|       npi|tin_type|      tin|
+-----------------+----------+--------+---------+
|         10001001|1235233008|     ein|043267217|
|         10001001|1316041189|     ein|043267217|
|         10001001|1780788554|     ein|043267217|
|         10001001|1891068409|     ein|043267217|
|         10001001|1366459570|     ein|111562701|
|         10001001|1417915653|     ein|113358535|
|         10001001|1417915653|     ein|133888838|
|         10002001|1609829761|     ein|000004110|
|         10002001|1821482241|     ein|000004110|
|         10002001|1760986277|     ein|006980743|
|         10002001|1215075882|     ein|010550744|
|         10002001|1013917665|     ein|010555304|
|         10002001|1679780811|     ein|010555483|
|         10002001|1700093952|     ein|010555483|
|         10002001|1780072447|     ein|010555483|
|         10002001|1952532970|     ein|010555483|
|         10002001|1376647511|     ein|010567880|


In [8]:
# rem_hyp.toPandas().to_json('in_prov.json', orient='records')

In [9]:
df = rem_hyp.withColumn('tin_type',
     when((col('tin_type')== 'ein'), 1).when((col('tin_type')== 'npi'), 2)

                       )

In [10]:
df.show()

+-----------------+----------+--------+---------+
|provider_group_id|       npi|tin_type|      tin|
+-----------------+----------+--------+---------+
|         10001001|1235233008|       1|043267217|
|         10001001|1316041189|       1|043267217|
|         10001001|1780788554|       1|043267217|
|         10001001|1891068409|       1|043267217|
|         10001001|1366459570|       1|111562701|
|         10001001|1417915653|       1|113358535|
|         10001001|1417915653|       1|133888838|
|         10002001|1609829761|       1|000004110|
|         10002001|1821482241|       1|000004110|
|         10002001|1760986277|       1|006980743|
|         10002001|1215075882|       1|010550744|
|         10002001|1013917665|       1|010555304|
|         10002001|1679780811|       1|010555483|
|         10002001|1700093952|       1|010555483|
|         10002001|1780072447|       1|010555483|
|         10002001|1952532970|       1|010555483|
|         10002001|1376647511|       1|010567880|


In [11]:
rename_col = df.withColumnRenamed('tin_value','tin')
rename_col.show()

+-----------------+----------+--------+---------+
|provider_group_id|       npi|tin_type|      tin|
+-----------------+----------+--------+---------+
|         10001001|1235233008|       1|043267217|
|         10001001|1316041189|       1|043267217|
|         10001001|1780788554|       1|043267217|
|         10001001|1891068409|       1|043267217|
|         10001001|1366459570|       1|111562701|
|         10001001|1417915653|       1|113358535|
|         10001001|1417915653|       1|133888838|
|         10002001|1609829761|       1|000004110|
|         10002001|1821482241|       1|000004110|
|         10002001|1760986277|       1|006980743|
|         10002001|1215075882|       1|010550744|
|         10002001|1013917665|       1|010555304|
|         10002001|1679780811|       1|010555483|
|         10002001|1700093952|       1|010555483|
|         10002001|1780072447|       1|010555483|
|         10002001|1952532970|       1|010555483|
|         10002001|1376647511|       1|010567880|


In [12]:
rename_col.printSchema()

root
 |-- provider_group_id: long (nullable = true)
 |-- npi: long (nullable = true)
 |-- tin_type: integer (nullable = true)
 |-- tin: string (nullable = true)



In [13]:
rate_cast = rename_col.withColumn("tin_type",col("tin_type").cast(ShortType()))

In [14]:
rate_cast.printSchema()

root
 |-- provider_group_id: long (nullable = true)
 |-- npi: long (nullable = true)
 |-- tin_type: short (nullable = true)
 |-- tin: string (nullable = true)



In [15]:

rate_cast.write.parquet('in_prov.parquet')



In [16]:
# open_df =  spark.read.parquet('in_prov.parquet')